In [9]:
import os
import dotenv
import requests

In [20]:
from dotenv import load_dotenv

load_dotenv()

A2E_API_TOKEN = os.getenv("A2E_API_TOKEN")

# Text-To-Image

In [14]:
avatar_description = """
A professional male avatar representing someone from the Emirates with a confident yet 
approachable presence. He has slightly short, neatly styled dark hair and warm hazel-brown 
eyes. His glasses are a modern rectangular frame, giving him a sharp and intellectual look. 
He wears a light stubble instead of a fuller beard, keeping the style clean. His expression 
conveys calmness, kindness, and reliability. He is dressed in a smart button-up shirt with rolled sleeves, 
suggesting professionalism with a touch of friendliness. His build is athletic with broad shoulders, 
making him appear strong, dependable, and knowledgeable."""

In [ ]:
import http.client
import json

conn = http.client.HTTPSConnection("video.a2e.ai")
payload = json.dumps({
   "name": "Test Name",
   "prompt": avatar_description,
   "req_key": "high_aes_general_v21_L",
   "width": 1024,
   "height": 768
})
headers = {
    "Authorization": f"Bearer {A2E_API_TOKEN}",
    "Content-Type": "application/json"
}
conn.request("POST", "/api/v1/userText2image/start", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{"code":0,"data":{"_id":"68b2ede4bab4c3003a19f5e9","name":"Test Name","prompt":"\nA professional male avatar representing someone from the Emirates with a confident yet \napproachable presence. He has slightly short, neatly styled dark hair and warm hazel-brown \neyes. His glasses are a modern rectangular frame, giving him a sharp and intellectual look. \nHe wears a light stubble instead of a fuller beard, keeping the style clean. His expression \nconveys calmness, kindness, and reliability. He is dressed in a smart button-up shirt with rolled sleeves, \nsuggesting professionalism with a touch of friendliness. His build is athletic with broad shoulders, \nmaking him appear strong, dependable, and knowledgeable.","image_urls":["https://30days-tos.avatar2everyone.com/adam2eve/stable/users/68b2daf50c05b8003b960b6d/faf8830a-2c5d-4590-86c5-1300e4c76b4e.jpeg"],"current_status":"completed","coins":10,"createdAt":"2025-08-30T12:26:12.841Z","failed_message":"","failed_code":"","width":1024,"hei

In [ ]:
image_url = data['data']['image_urls'][0]

In [27]:
import json
import requests
from pathlib import Path

# 1) Parse the JSON bytes you showed earlier
raw = b'{"code":0,"data":{"_id":"68b2ede4bab4c3003a19f5e9","name":"Test Name","prompt":"\\nA professional male avatar...","image_urls":["https://30days-tos.avatar2everyone.com/adam2eve/stable/users/68b2daf50c05b8003b960b6d/faf8830a-2c5d-4590-86c5-1300e4c76b4e.jpeg"],"current_status":"completed"},"trace_id":"..."}'
data = json.loads(raw.decode("utf-8"))

# 2) Extract a plain string URL (not a list)
urls = data["data"]["image_urls"]
url = urls[0] if isinstance(urls, list) else urls   # ensure it's a str

# 3) Download with requests (streaming, timeout, simple headers)
resp = requests.get(url, stream=True, timeout=30, headers={"User-Agent": "python-requests"})
resp.raise_for_status()  # will raise if not 2xx

# 4) Pick a filename (use extension from URL or Content-Type)
ext = Path(url).suffix or ".jpg"
out_path = Path(f"avatar{ext}")

with open(out_path, "wb") as f:
    for chunk in resp.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)

print(f"Saved -> {out_path.resolve()}")


Saved -> D:\NETIX\SwarmForge\Ai Avatar\avatar.jpeg


---

# Create Avatar From Image

In [ ]:
# image_url = "https://30days-tos.avatar2everyone.com/adam2eve/stable/users/68b2daf50c05b8003b960b6d/faf8830a-2c5d-4590-86c5-1300e4c76b4e.jpeg"

In [ ]:
import os
import requests

def create_avatar_from_image(name: str, gender: str, image_url: str,
                             bg_color: str | None = None,
                             bg_image_url: str | None = None,
                             model_version: str | None = None,
                             prompt: str | None = None,
                             negative_prompt: str | None = None,
                             skip_preview: bool | None = None):
    """
    Creates a custom avatar using an IMAGE (costs credits, per A2E docs).
    Either image_url or video_url must be provided; here we use image_url.
    """
    if " " in image_url:
        raise ValueError("image_url must not contain spaces; URL-encode them as %20 if needed.")

    url = f"{A2E_BASE}/api/v1/userVideoTwin/startTraining"

    headers = {
        "Authorization": f"Bearer {A2E_API_TOKEN or 'REPLACE_ME'}",
        "x-lang": "en-US",                 # UI language of the platform, not the avatar voice
        "Content-Type": "application/json"
    }

    body = {
        "name": name,
        "gender": gender,                  # "female" or "male"
        "image_url": image_url
        # do NOT include video_url when using image_url
    }

    # Optional fields (per docs)
    if bg_color:        # API field name is spelled 'video_backgroud_color' in docs
        body["video_backgroud_color"] = bg_color  # e.g., "rgb(61,165,82)"
    if bg_image_url:
        body["video_backgroud_image"] = bg_image_url
    if model_version in {"V2.1", "V2.0"}:
        body["model_version"] = model_version
    if prompt:
        body["prompt"] = prompt
    if negative_prompt:
        body["negative_prompt"] = negative_prompt
    if skip_preview is not None:
        body["skipPreview"] = bool(skip_preview)

    resp = requests.post(url, json=body, headers=headers, timeout=60)
    resp.raise_for_status()
    data = resp.json()

    # A successful response (code==0) includes the record under data; grab a few useful fields:
    if data.get("code") == 0 and isinstance(data.get("data"), dict):
        d = data["data"]
        print("Avatar training created.")
        print(f"  id (user_video_twin_id): {d.get('_id')}")
        print(f"  name: {d.get('name')}")
        print(f"  gender: {d.get('gender')}")
        print(f"  current_status: {d.get('current_status')}")
        print(f"  image_url: {d.get('image_url')}")
        print(f"  video_backgroud_color: {d.get('video_backgroud_color')}")
        # d.get('task_id') may be empty initially; use the avatar list/status endpoints to track progress
    else:
        print("Unexpected response:", data)

    return data


In [54]:
image_url

'https://30days-tos.avatar2everyone.com/adam2eve/stable/users/68b2daf50c05b8003b960b6d/faf8830a-2c5d-4590-86c5-1300e4c76b4e.jpeg'

In [55]:
# Example usage — replace with your values:
result = create_avatar_from_image(
    name="Second Emirate Guy",
    gender="male",
    image_url=image_url
)

Avatar training created.
  id (user_video_twin_id): 68b30362bf45570042defd0d
  name: Second Emirate Guy
  gender: male
  current_status: initialized
  image_url: https://30days-tos.avatar2everyone.com/adam2eve/stable/users/68b2daf50c05b8003b960b6d/faf8830a-2c5d-4590-86c5-1300e4c76b4e.jpeg
  video_backgroud_color: 


In [57]:
AVATAR_ID = result['data']['_id']
AVATAR_ID

'68b30362bf45570042defd0d'

In [ ]:
import http.client
import json

def get_user_video_twin(a2e_token: str, avatar_id: str) -> dict:
    """
    Fetch user video twin data from A2E API.

    Args:
        a2e_token (str): Authorization token for the API.
        avatar_id (str): ID of the avatar to fetch.

    Returns:
        dict: Parsed JSON response from the API.
    """
    conn = http.client.HTTPSConnection("video.a2e.ai")
    headers = {
        "Authorization": f"Bearer {A2E_API_TOKEN}",
        "x-lang": "en-US"
    }
    
    conn.request("GET", f"/api/v1/userVideoTwin/{avatar_id}", body="", headers=headers)
    res = conn.getresponse()
    data = res.read()
    
    conn.close()
    return json.loads(data.decode("utf-8"))


In [1]:
def wait_until_ready(avatar_id: str, timeout_s: int = 1800, poll_s: int = 20):
    """
    Polls until current_status is a terminal 'ready/success' or fails/times out.
    """
    start = time.time()

    while True:
        item = get_user_video_twin(A2E_API_TOKEN, AVATAR_ID)
        status = item['data']['current_status']
        print("status:", status)
        if status in {"ready", "trained", "succeeded", "completed"}:
            return item
        if status in {"failed", "error"}:
            raise RuntimeError(f"Training failed: {item}")
        if time.time() - start > timeout_s:
            raise TimeoutError(f"Timed out waiting for avatar {avatar_id} to be ready")
        time.sleep(poll_s)

In [ ]:
import time

import time

def wait_until_ready(avatar_id: str, timeout_s: int = 600, poll_s: int = 20):
    start = time.time()

    while True:
        # Make sure to pass the avatar_id, not a global AVATAR_ID
        item = get_user_video_twin(A2E_API_TOKEN, avatar_id)
        status = item['data']['current_status']
        print(f"status: {status}")

        if status in {"ready", "trained", "succeeded", "completed"}:
            return item

        if status in {"failed", "error"}:
            raise RuntimeError(f"Training failed: {item}")

        if time.time() - start > timeout_s:
            raise TimeoutError(f"Timed out waiting for avatar {avatar_id} to be ready")

        time.sleep(poll_s)


In [71]:
avatar_info = wait_until_ready(AVATAR_ID)

status: completed


In [72]:
avatar_info

{'code': 0,
 'data': {'_id': '68b30362bf45570042defd0d',
  'name': 'Second Emirate Guy',
  'gender': 'male',
  'image_url': 'https://30days-tos.avatar2everyone.com/adam2eve/stable/users/68b2daf50c05b8003b960b6d/faf8830a-2c5d-4590-86c5-1300e4c76b4e.jpeg',
  'video_url': '',
  'current_status': 'completed',
  'failed_code': '0',
  'failed_message': None,
  'wl_model': 'live_protrait',
  'close_mouth_path': '',
  'task_id': '',
  'task_status': '',
  'task_result': {},
  'video_backgroud_image': '',
  'video_backgroud_color': '',
  'image_result_url': 'https://prod-tos.avatar2everyone.com/adam2eve/stable/user/68b2daf50c05b8003b960b6d/userVideoTwin/68b30362bf45570042defd0d/f29d4989-3b05-4470-8f91-53989d8839bb.mp4',
  'image_error_code': '',
  'preview_result_url': 'https://prod-tos.avatar2everyone.com/adam2eve/stable/user/68b2daf50c05b8003b960b6d/userVideoTwin/68b30362bf45570042defd0d/preview_1756562491564.mp4',
  'version': 1,
  'isSilent': True,
  'hasVoiceClone': False,
  'hasVideoClone

### Continute lip-syncing training

In [32]:
avatar_id = '68b30362bf45570042defd0d'

In [7]:
import http.client
import json

conn = http.client.HTTPSConnection("video.a2e.ai")
payload = json.dumps({
   "_id": avatar_id
})
headers = {
    "Authorization": f"Bearer {A2E_API_TOKEN}",
    "Content-Type": "application/json"
}
conn.request("POST", "/api/v1/userVideoTwin/continueTranining", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{"code":1000,"msg":"Image does not support continue training","trace_id":"a0bd1a30-e609-44c1-88c8-6c5090cb40cf"}


---

# Creating Video With The Avatar

### Creating Text-To-Speech

In [ ]:
import requests
import pprint

url = "https://video.a2e.ai/api/v1/anchor/voice_list"
headers = {
    "Authorization": f"Bearer {A2E_API_TOKEN}",
    "Accept": "application/json",
    "User-Agent": "a2e-client/1.0"
}
params = {
    "country": "en",
    "region": "US",
    "voice_map_type": "en-US"
}

r = requests.get(url, headers=headers, params=params, timeout=30)
print("Status:", r.status_code, r.reason)
if r.ok:
    pprint.pprint(json.loads(r.text))

Status: 200 OK
{'code': 0,
 'data': [{'children': [{'label': 'Aria - Sexy Female Villain Voice',
                         'ttsRate': 3,
                         'value': '66ca504f2732c24634224075'},
                        {'label': 'ASMR Dr Lovejoy',
                         'ttsRate': 3,
                         'value': '66dc6764547f617d734c4657'},
                        {'label': 'august',
                         'ttsRate': 3,
                         'value': '66c67a178d721777dc7150c6'},
                        {'label': 'Ava Multilingual',
                         'ttsRate': 1,
                         'value': '66d3fa4ab76478b1272f2249'},
                        {'label': 'Emma Multilingual',
                         'ttsRate': 1,
                         'value': '66d3fb89357648b14c4f4f26'},
                        {'label': 'Jessica',
                         'ttsRate': 3,
                         'value': '669b7015ba04042f4090b288'},
                        {'label': 'Laura

In [ ]:
import requests
import json


def generate_tts(msg, voice_id, country="en", region="US", speech_rate=1.0):
    """Generate TTS Audio (Text-to-Speech) → returns audioSrc URL."""

    url = "https://video.a2e.ai/api/v1/video/send_tts"

    body = {
        "msg": msg,
        "tts_id": voice_id,
        "speechRate": speech_rate,
        "country": country,
        "region": region,
    }

    headers = {
        "Authorization": f"Bearer {A2E_API_TOKEN}",
        "Content-Type": "application/json",
        "Accept": "application/json",
        "User-Agent": "a2e-client/1.0",
    }

    r = requests.post(url, headers=headers, json=body, timeout=60)
    r.raise_for_status()

    try:
        j = r.json()
    except ValueError:
        # response is not JSON
        return r.text

    return j.data


In [ ]:
voice_id = "6625ebd4613f49985c349f95"

In [ ]:
script = "Hello, I'm your content creator assistant. I will help you through the process of creating and publishing your content."

In [ ]:
# audio_src = generate_tts(script, voice_id)

AttributeError: 'str' object has no attribute 'get'

In [ ]:
audio_src = 'https://1day-tos.a2e.com.cn/tts3party_cache/stable/el/Brian/speed/a9a730f81d8281644d6a183a949fa7a0.wav'

### Creating Video With Voice and Avatar

In [17]:
import http.client
import json

# Step 1: Get the list of available anchors
conn = http.client.HTTPSConnection("video.a2e.ai")
headers = {
    "Authorization": f"Bearer {A2E_API_TOKEN}",
    "Content-Type": "application/json"
}
conn.request("GET", "/api/v1/anchor/character_list", headers=headers)
res = conn.getresponse()
data = res.read()
anchors = json.loads(data.decode("utf-8"))

In [34]:
avatar_id = '68b30362bf45570042defd0d'

In [35]:
import http.client

conn = http.client.HTTPSConnection("video.a2e.ai")
payload = ''
headers = {
    "Authorization": f"Bearer {A2E_API_TOKEN}",
    "Content-Type": "application/json"
}
conn.request("GET", f"/api/v1/anchor/character_list?user_video_twin_id={avatar_id}&type=custom", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{"code":0,"data":[{"name":"Second Emirate Guy","type":"custom","_id":"68b3043e246274004268a84e","createdAt":"2025-08-30T14:01:34.513Z","width":0,"height":0,"isExclusive":true,"wl_model":"live_protrait","user_photo_twin_id":"","user_photo_twin_image_id":"","user_video_twin_id":"68b30362bf45570042defd0d","user_video_twin_training_model":"fast","background_img":"","background_color":"","base_video":"https://prod-tos.avatar2everyone.com/adam2eve/stable/user/68b2daf50c05b8003b960b6d/userVideoTwin/68b30362bf45570042defd0d/f29d4989-3b05-4470-8f91-53989d8839bb.mp4","people_img":"","lang":[],"video_cover":"https://prod-tos.avatar2everyone.com/adam2eve/stable/custom_avatar_cover/cdfd5403-e361-47c1-b1f9-4bf2093e5ba7.jpeg"}],"trace_id":"e5182b11-5a65-46a2-970b-b91ec25ad1f2"}


In [43]:
anchor_id = json.loads(data.decode("utf-8"))['data'][0]["_id"]
anchor_id

'68b3043e246274004268a84e'

In [44]:
import http.client
import json

conn = http.client.HTTPSConnection("video.a2e.ai")
payload = json.dumps({
   "title": "Second Emirate Guy Greeting Video",
   "anchor_id": anchor_id,
   "anchor_type": 1,
   "audioSrc": audio_src,
#    "msg": script,
   "resolution": 1080,
   "web_bg_width": 853,
   "web_bg_height": 480,
   "web_people_width": 270,
   "web_people_height": 480,
   "web_people_x": 292,
   "web_people_y": 0,
   "isSkipRs": True,
   "isCaptionEnabled": False,

})
headers = {
    "Authorization": f"Bearer {A2E_API_TOKEN}",
    "Content-Type": "application/json"
}
conn.request("POST", "/api/v1/video/generate", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{"code":0,"data":{"title":"Second Emirate Guy Greeting Video","msg":"","algo_result":"","result":"","retry":0,"process":0,"isDel":false,"deletedByAccount":false,"isSkipRs":true,"status":"init","error":"","error_code":"","anchor_type":"1","resolution":"1080","voice_type":"0","audioSrc":"https://1day-tos.a2e.com.cn/tts3party_cache/stable/el/Brian/speed/a9a730f81d8281644d6a183a949fa7a0.wav","custom_voice":"","priority":3,"end_success_date":null,"restart_date":null,"audioLen":7,"web_people_max_width":null,"web_people_max_height":null,"web_people_width":608,"web_people_height":1080,"web_people_x":657,"web_people_y":0,"web_bg_width":1920,"web_bg_height":1080,"color":"","result_cover":"","wl_model":"","anchor_background_img":"","anchor_background_color":"","isSkipGp":true,"isAllowReverse":true,"isCaptionEnabled":false,"jah_call_time":null,"isToPublicPool":false,"lip_model":"","hasRefundCoin":false,"video_cover":"https://prod-tos.avatar2everyone.com/adam2eve/stable/custom_avatar_cover/cdfd5403

In [46]:
from pprint import pprint

pprint(json.loads(data.decode("utf-8")))

{'code': 0,
 'data': {'_id': '68b421ef279d69003c253eda',
          'algo_result': '',
          'anchor_background_color': '',
          'anchor_background_img': '',
          'anchor_id': '68b3043e246274004268a84e',
          'anchor_type': '1',
          'audioLen': 7,
          'audioSrc': 'https://1day-tos.a2e.com.cn/tts3party_cache/stable/el/Brian/speed/a9a730f81d8281644d6a183a949fa7a0.wav',
          'color': '',
          'copy_retry_count': 0,
          'createdAt': '2025-08-31T10:20:31.137Z',
          'custom_voice': '',
          'dailyIndex': 1,
          'deletedByAccount': False,
          'end_success_date': None,
          'error': '',
          'error_code': '',
          'hasRefundCoin': False,
          'host': 'video.a2e.ai',
          'isAllowReverse': True,
          'isApi': True,
          'isCaptionEnabled': False,
          'isDel': False,
          'isSkipGp': True,
          'isSkipRs': True,
          'isToPublicPool': False,
          'jah_call_time': None

In [48]:
video_task_id = json.loads(data.decode("utf-8"))['data']['_id']

In [50]:
import http.client
import json

conn = http.client.HTTPSConnection("video.a2e.ai")
payload = json.dumps({
   "_id": video_task_id
})
headers = {
    "Authorization": f"Bearer {A2E_API_TOKEN}",
    "Content-Type": "application/json"
}
conn.request("POST", "/api/v1/video/awsResult", payload, headers)
res = conn.getresponse()
data = res.read()
pprint(data.decode("utf-8"))

('{"code":0,"data":[{"_id":"68b421ef279d69003c253eda","title":"Second Emirate '
 'Guy Greeting '
 'Video","msg":"","result":"https://30days-tos.avatar2everyone.com/adam2eve/stable/users/68b2daf50c05b8003b960b6d/video_result/68b421ef279d69003c253eda/Second%20Emirate%20Guy%20Greeting%20Video.mp4","process":100,"status":"success","anchor_type":"1","end_success_date":"2025-08-31T10:22:11.876Z","web_bg_width":1920,"web_bg_height":1080,"color":"","result_cover":"","video_cover":"https://prod-tos.avatar2everyone.com/adam2eve/stable/custom_avatar_cover/cdfd5403-e361-47c1-b1f9-4bf2093e5ba7.jpeg","anchor_id":"68b3043e246274004268a84e","createdAt":"2025-08-31T10:20:31.137Z","diff_s":199,"current_date":"2025-08-31 '
 '18:23:50"}],"trace_id":"4cedcad7-4e12-4f04-91a6-106ff1254224"}')
